In [1]:
import os
import csv

In [4]:
prefix_prompt = 'Solve the following and respond with only the final answer:'

In [ ]:
#original
function = lambda a,b: a + b
correct_prompt_templates = lambda n1, n2: f'What is {n1} plus {n2}?'

In [ ]:
#Counter factual
function = lambda a,b: a + 2 * b
correct_prompt_templates = lambda n1, n2: f'What is {n1} plus {n2}?'

In [10]:
#New logic
function = lambda a,b: a + 2 * b
correct_prompt_templates = lambda n1, n2: f'What is zarp({n1},{n2})='

In [11]:
import random

# Set target dataset sizes
total_examples = 6000
train_size = int(0.9 * total_examples)  # 5400
test_size = total_examples - train_size  # 600

# Generate and shuffle all possible two-digit number pairs
all_combos = [(a, b) for a in range(10, 100) for b in range(10, 100)]
random.shuffle(all_combos)

# Storage
results_train = [['clean', 'corrupted', 'answer']]
results_test = [['clean', 'corrupted', 'answer']]

train_count = 0
test_count = 0

# Loop through combos and generate examples
for a, b in all_combos:
    if train_count >= train_size and test_count >= test_size:
        break  # Stop when both sets are full

    answer = function(a, b)
    if answer <= 0:
        continue  # Skip invalid answers

    c = random.randint(10, 99)
    while c == b:
        c = random.randint(10, 99)

    prompt = correct_prompt_templates(a, b)
    corrupted_prompt = correct_prompt_templates(a, c)

    clean_prompt = prefix_prompt + ' ' + prompt
    corrupted_prompt = prefix_prompt + ' ' + corrupted_prompt

    if train_count < train_size:
        results_train.append([clean_prompt, corrupted_prompt, answer])
        train_count += 1
    elif test_count < test_size:
        results_test.append([clean_prompt, corrupted_prompt, answer])
        test_count += 1


In [12]:
output_directory = 'new-zarp/datasets_csv/'

os.makedirs(output_directory, exist_ok=True)

filename_train = os.path.join(output_directory, "train.csv")
filename_test = os.path.join(output_directory, "validation.csv")

with open(filename_train, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results_train)

with open(filename_test, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results_test)

In [33]:
import random
import csv

def generate_unique_problems(num_problems, val_split=0.2):
    problems = set()
    num_each = num_problems // 2

    while len(problems) < num_problems:
        a = random.randint(10, 99)
        b = random.randint(10, 99)

        # Balance the number of '+' and '-' problems
        if len([p for p in problems if '+' in p[0]]) < num_each:
            op = '+'
        else:
            op = '-'
            if b > a:
                a, b = b, a  # Ensure a - b is non-negative

        # Create the prompt and compute the result
        prompt = f"Solve the following and respond with only the final answer: {a}{op}{b}="
        result = eval(f"{a} {op} {b}")

        # ✅ Only add if result is positive
        if result > 0:
            problems.add((prompt, result, a, b, op))

    problems = list(problems)
    random.shuffle(problems)

    split_idx = int((1 - val_split) * len(problems))
    train = problems[:split_idx]
    val = problems[split_idx:]

    return train, val

def corrupt_second_number(a, b, op):
    # Always change the second number (b)
    new_b = random.choice([x for x in range(10, 100) if x != b])
    corrupted = f"Solve the following and respond with only the final answer: {a}{op}{new_b}="
    return corrupted

def write_csv(filename, dataset):
    with open(filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["clean", "corrupted", "answer"])
        for prompt, answer, a, b, op in dataset:
            corrupted = corrupt_second_number(a, b, op)
            writer.writerow([prompt, corrupted, answer])

# === Example usage ===
num_total = 6000
train_set, val_set = generate_unique_problems(num_total, val_split=0.1)

write_csv("addsub/datasets_csv/train.csv", train_set)
write_csv("addsub/datasets_csv/validation.csv", val_set)

print("✅ CSV files created: train.csv and val.csv")


✅ CSV files created: train.csv and val.csv


In [32]:
import random
import csv

def generate_unique_problems(num_problems, val_split=0.2):
    problems = set()
    num_each = num_problems // 2

    while len(problems) < num_problems:
        a = random.randint(10, 99)
        b = random.randint(10, 99)

        # Choose operation to balance + and -
        if len([p for p in problems if '+' in p[0]]) < num_each:
            op = '+'
        else:
            op = '-'
            if b > a:
                a, b = b, a  # Ensure a - b is not negative

        prompt = f"Solve the following and respond with only the final answer: {a}{op}{b}="

        # Flip the meaning: + becomes -, - becomes +
        if op == '+':
            result = a - b
        else:
            result = a + b

        # ✅ Only keep examples with positive results
        if result > 0:
            problems.add((prompt, result, a, b, op))

    problems = list(problems)
    random.shuffle(problems)

    split_idx = int((1 - val_split) * len(problems))
    train = problems[:split_idx]
    val = problems[split_idx:]

    return train, val

def corrupt_second_number(a, b, op):
    # Always change the second number (b)
    new_b = random.choice([x for x in range(10, 100) if x != b])
    corrupted = f"Solve the following and respond with only the final answer: {a}{op}{new_b}="
    return corrupted

def write_csv(filename, dataset):
    with open(filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["clean", "corrupted", "answer"])
        for prompt, answer, a, b, op in dataset:
            corrupted = corrupt_second_number(a, b, op)
            writer.writerow([prompt, corrupted, answer])

# === Example usage ===
num_total = 6000
train_set, val_set = generate_unique_problems(num_total, val_split=0.1)

write_csv("addsub_inverse/datasets_csv/train.csv", train_set)
write_csv("addsub_inverse/datasets_csv/validation.csv", val_set)

print("✅ CSV files created: train.csv and val.csv")


✅ CSV files created: train.csv and val.csv
